Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [1]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df1 = pd.DataFrame.from_records(ham)
df2 = pd.DataFrame.from_records(spam)
df = pd.concat([df1, df2], ignore_index=True)

skipped 2649.2004-10-27.GP.spam.txt
skipped 0754.2004-04-01.GP.spam.txt
skipped 2042.2004-08-30.GP.spam.txt
skipped 3304.2004-12-26.GP.spam.txt
skipped 4142.2005-03-31.GP.spam.txt
skipped 3364.2005-01-01.GP.spam.txt
skipped 4201.2005-04-05.GP.spam.txt
skipped 2140.2004-09-13.GP.spam.txt
skipped 2248.2004-09-23.GP.spam.txt
skipped 4350.2005-04-23.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 1414.2004-06-24.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 5105.2005-08-31.GP.spam.txt


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [2]:
import re

def preprocessor(e):
    return re.sub('[^A-Za-z]', ' ', e).lower()

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

vectorizer = CountVectorizer(preprocessor=preprocessor)
X_train, X_test, y_train, y_test = train_test_split(df['content'],df['category'], test_size=0.2, random_state=42)
X_train_df = vectorizer.fit_transform(X_train)

model = LogisticRegression()
model.fit(X_train_df, y_train)

X_test_df = vectorizer.transform(X_test)
y_pred = model.predict(X_test_df)

print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}\n')
print(f'Detailed Statistics: \n{classification_report(y_test, y_pred)}')


Confusion Matrix: 
[[693  14]
 [ 11 314]]

Detailed Statistics: 
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       707
        spam       0.96      0.97      0.96       325

    accuracy                           0.98      1032
   macro avg       0.97      0.97      0.97      1032
weighted avg       0.98      0.98      0.98      1032



Step 4.

In [4]:
features = vectorizer.get_feature_names_out()
importance = model.coef_[0]

indexed_importance = list(enumerate(importance))
sorted_importance = sorted(indexed_importance, key=lambda x: abs(x[1]), reverse=True)

top_positive = [(index, importance) for index, importance in sorted_importance if importance > 0][:10]
top_negative = [(index, importance) for index, importance in sorted_importance if importance < 0][:10]

df_top_positive = pd.DataFrame(top_positive, columns=['Feature Index', 'Importance'])
df_top_negative = pd.DataFrame(top_negative, columns=['Feature Index', 'Importance'])

df_top_positive['Feature'] = [features[index] for index, _ in top_positive]
df_top_negative['Feature'] = [features[index] for index, _ in top_negative]

print('Top 10 Positive Features')
print(df_top_positive)

print('\nTop 10 Negative Features')
print(df_top_negative)

Top 10 Positive Features
   Feature Index  Importance  Feature
0          17754    0.894404     http
1          17019    0.814249    hello
2          30340    0.780498  removed
3          28501    0.773066   prices
4          25157    0.757636       no
5          24007    0.745125    money
6          17082    0.706949     here
7          25961    0.659537     only
8          26564    0.647642     pain
9          24107    0.635354     more

Top 10 Negative Features
   Feature Index  Importance   Feature
0           2558   -1.691388  attached
1          12606   -1.555385     enron
2           9459   -1.429007     daren
3          35650   -1.332251    thanks
4          11028   -1.285912       doc
5           9641   -1.237547      deal
6          23381   -1.115141     meter
7          17678   -1.108359       hpl
8          24834   -1.000763      neon
9          39776   -0.951037       xls
